In [ ]:
import multiprocessing as mp
from multiprocessing import Pool, cpu_count

import numpy as np 
from scipy.special import gammaincinv, gamma, factorial, lambertw

from astropy.modeling import models 
from astropy.table import Table, vstack
from astropy.modeling import models, fitting
from astropy.modeling import custom_model, powerlaws, functional_models
from astropy.utils.console import ProgressBar
from astropy.convolution import convolve

from petrofit.models import sersic_enclosed, sersic_enclosed_inv, petrosian_profile, sersic_enclosed, make_grid, PSFModel
from petrofit.petrosian import Petrosian, PetrosianCorrection, calculate_petrosian,discrete_petrosian_r
from petrofit.fitting import model_to_image, fit_model
from petrofit.photometry import calculate_photometic_density,  make_radius_list
from petrofit.petrosian import Petrosian
from petrofit.utils import closest_value_index, get_interpolated_values
from scipy.interpolate import interp1d
from petrofit.petrosian import fraction_flux_to_r

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

# Generate or Load Table

## Old Generator

In [ ]:
# run = 0

# delta = 20
# r_eff_list = [50]#np.arange(delta, 100 + delta, delta)
# delta = 0.01
# n_list = np.arange(0.2, 20 + delta, delta)

# rows = []
# if run:
#     with ProgressBar(len(r_eff_list) * len(n_list), ipython_widget=True) as bar:
#         for n_idx, n in enumerate(n_list):

#             for r_eff_idx, r_eff in enumerate(r_eff_list):
#                 bar.update()

#                 b_n = gammaincinv(2. * n, 0.5)
#                 amplitude=1 #/np.exp(b_n)

#                 total_flux = sersic_enclosed(np.inf, amplitude, r_eff, n)
#                 enclosed_99 = total_flux*.99

#                 r_99 = sersic_enclosed_inv(enclosed_99, amplitude, r_eff, n)
#                 r_20 = sersic_enclosed_inv(enclosed_99*0.2, amplitude, r_eff, n)
#                 r_80 = sersic_enclosed_inv(enclosed_99*0.8, amplitude, r_eff, n)
#                 r_90 = sersic_enclosed_inv(enclosed_99*0.9, amplitude, r_eff, n)
#                 r_50 = sersic_enclosed_inv(enclosed_99*0.5, amplitude, r_eff, n)

#                 oversample = 10000
#                 compute_r_cutoff = 2000
                
#                 if r_99 > compute_r_cutoff: 
#                     max_pix=int(r_90)*oversample
#                 else:
#                     max_pix=int(r_99*2)*oversample

#                 r_list = make_radius_list(
#                             max_pix=max_pix, # Max pixel to go up to
#                             n=max_pix,# the number of radii to produce
#                 )
#                 r_list /=oversample
                
#                 if r_99 > compute_r_cutoff: 
#                     delta = ((3*r_99)-r_90)
#                     if delta > 1000:
#                         delta /= 1000
#                     else:
#                         delta = 1

#                     r_list = list(r_list) + list(np.arange(r_90+delta, (3*r_99)+delta, delta))
#                     r_list = np.array(r_list)

#                 flux_arr = sersic_enclosed(r_list, amplitude, r_eff, n)
#                 area_arr = np.pi*(r_list**2)
                
#                 petrosian_list = petrosian_profile(r_list, r_eff, n)
                
#                 # Interpolate values
                
#                 eta = 0.2 
#                 idx = closest_value_index(eta, petrosian_list)
#                 r_petrosian = r_list[idx]
                
#                 idx = closest_value_index(0.2, petrosian_list)
#                 r_pertro_02 = r_list[idx]
                
#                 idx = closest_value_index(0.5, petrosian_list)
#                 r_pertro_05 = r_list[idx]
                
#                 c_petro_0205 = 5 * np.log(r_pertro_05 / r_pertro_02)
                
#                 #r_list_new, petrosian_list_new = get_interpolated_values(r_list, petrosian_list, num=len(r_list)*10, kind='linear')
#                 #idx = closest_value_index(eta, petrosian_list_new)
#                 #r_petrosian = r_list_new[idx]
                
#                 #f = interp1d(petrosian_list, r_list, kind='linear')
#                 #r_petrosian = f(eta)
                
#                 idx_list = np.where(petrosian_list <= eta)[0]

#                 #print(total_flux)
#                 epsilon = r_99 / r_petrosian
#                 p_corr = None#Petrosian(r_list, area_arr, flux_arr, epsilon=epsilon)
      
#                 p_corr_eta = eta
#                 p_corr_epsilon = epsilon
#                 p_corr_r_petrosian = r_petrosian
#                 p_corr_r_total_flux = r_petrosian * p_corr_epsilon
#                 f = interp1d(r_list, flux_arr, kind='linear')
#                 p_corr_total_flux = f(p_corr_r_total_flux)
#                 p_corr_r_20 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.2)
#                 p_corr_r_80 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.8)
#                 p_corr_r_half_light = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.5)
                
                                
#                 p_uncorr_eta = eta
#                 p_uncorr_epsilon = 2
#                 p_uncorr_r_petrosian = r_petrosian
#                 p_uncorr_r_total_flux = r_petrosian * p_uncorr_epsilon
#                 f = interp1d(r_list, flux_arr, kind='linear')
#                 p_uncorr_total_flux = f(p_uncorr_r_total_flux)
#                 p_uncorr_r_20 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.2)
#                 p_uncorr_r_80 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.8)
#                 p_uncorr_r_half_light = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.5)

#                 rows.append([
#                     n, r_eff, amplitude, total_flux, r_99, r_20, r_80, 5 *np.log10(r_80 / r_20), c_petro_0205,
#                     p_corr_eta, p_corr_epsilon, p_corr_r_petrosian, p_corr_r_half_light, p_corr_r_total_flux, p_corr_total_flux, p_corr_r_20, p_corr_r_80, 5 *np.log10(p_corr_r_80 / p_corr_r_20), 
#                     p_uncorr_eta, p_uncorr_epsilon, p_uncorr_r_petrosian, p_uncorr_r_half_light, p_uncorr_r_total_flux, p_uncorr_total_flux, p_uncorr_r_20, p_uncorr_r_80, 5 *np.log10(p_uncorr_r_80 / p_uncorr_r_20),
#                 ])

                

#                 assert not np.any(np.isnan(np.array(rows[-1]))), "{}".format(n)
#                 #plt.plot(r_list, petrosian_list)
#                 #plt.show()

#     names = [
#         "sersic_n", "sersic_r_eff", "sersic_amplitude", "sersic_total_flux_99", "sersic_r_99", "sersic_r_20", "sersic_r_80", "sersic_c2080", "sersic_petro_0205",
#         "corr_eta", "corr_epsilon", "corr_r_petrosian", "corr_r_half_light", "corr_r_total_flux", "corr_total_flux", "corr_r_20", "corr_r_80", "corr_c2080",
#         'uncorr_eta', 'uncorr_epsilon', 'uncorr_r_petrosian', 'uncorr_r_half_light', 'uncorr_r_total_flux', 'uncorr_total_flux', 'uncorr_r_20', 'uncorr_r_80', 'uncorr_c2080'
#     ]

#     t = Table(rows=rows, names=names)

#     plt.show()
#     print("done")
# else:
# #     t = Table.read('pure_grid_fine.csv')
#     t = Table.read('fine_petro_table_no_psf.csv')


In [ ]:
run = 0

delta = 20
r_eff_list = [50] 

# r_eff_list = np.arange(delta, 100 + delta, delta)

# Fine set
delta = 0.01
n_list = np.arange(0.1, 20 + delta, delta)

# Macro set
# delta = 0.5
# n_list = np.arange(0.5, 20 + delta, delta)

names = [
    "sersic_n", "sersic_r_eff", "sersic_amplitude", "sersic_total_flux_99", "sersic_r_99", "sersic_r_20", "sersic_r_80", "sersic_r_50", "sersic_r_90", "sersic_c2080", "sersic_c5090", "sersic_petro_0502",
    "corr_eta", "corr_epsilon", "corr_r_petrosian", "corr_r_half_light", "corr_r_total_flux", "corr_total_flux", "corr_r_20", "corr_r_80", "corr_r_50", "corr_r_90", "corr_c2080", "corr_c5090",
    'uncorr_eta', 'uncorr_epsilon', 'uncorr_r_petrosian', 'uncorr_r_half_light', 'uncorr_r_total_flux', 'uncorr_total_flux', 'uncorr_r_20', 'uncorr_r_80',  "uncorr_r_50", "uncorr_r_90", "uncorr_c2080", "uncorr_c5090",
]

def process_image(n):
    pid = mp.current_process().pid
    rows = []
    n = np.round(n, 4)
    #with ProgressBar(len(r_eff_list), ipython_widget=True) as bar:
    if True:
        for r_eff_idx, r_eff in enumerate(r_eff_list):
            
            b_n = gammaincinv(2. * n, 0.5)
            amplitude=1 #/np.exp(b_n)

            total_flux = sersic_enclosed(np.inf, amplitude, r_eff, n)
            enclosed_99 = total_flux*.99

            r_99 = sersic_enclosed_inv(enclosed_99, amplitude, r_eff, n)
            r_20 = sersic_enclosed_inv(enclosed_99*0.2, amplitude, r_eff, n)
            r_80 = sersic_enclosed_inv(enclosed_99*0.8, amplitude, r_eff, n)
            r_90 = sersic_enclosed_inv(enclosed_99*0.9, amplitude, r_eff, n)
            r_50 = sersic_enclosed_inv(enclosed_99*0.5, amplitude, r_eff, n)

            oversample = 10000
            compute_r_cutoff = 2000

            if r_99 > compute_r_cutoff: 
                max_pix=int(r_90)*oversample
            else:
                max_pix=int(r_99*2)*oversample

            r_list = make_radius_list(
                        max_pix=max_pix, # Max pixel to go up to
                        n=max_pix,# the number of radii to produce
            )
            r_list /=oversample

            if r_99 > compute_r_cutoff: 
                delta = ((3*r_99)-r_90)
                if delta > 1000:
                    delta /= 1000
                else:
                    delta = 1

                r_list = list(r_list) + list(np.arange(r_90+delta, (3*r_99)+delta, delta))
                r_list = np.array(r_list)

            flux_arr = sersic_enclosed(r_list, amplitude, r_eff, n)
            #area_arr = np.pi*(r_list**2)

            petrosian_list = petrosian_profile(r_list, r_eff, n)

            # Interpolate values

            eta = 0.2 
            idx = closest_value_index(eta, petrosian_list)
            r_petrosian = r_list[idx]

            idx = closest_value_index(0.2, petrosian_list)
            r_pertro_02 = r_list[idx]

            idx = closest_value_index(0.5, petrosian_list)
            r_pertro_05 = r_list[idx]

            c_petro_0502 = 5 * np.log10(r_pertro_02/r_pertro_05)
            
            #r_list_new, petrosian_list_new = get_interpolated_values(r_list, petrosian_list, num=len(r_list)*10, kind='linear')
            #idx = closest_value_index(eta, petrosian_list_new)
            #r_petrosian = r_list_new[idx]

            #f = interp1d(petrosian_list, r_list, kind='linear')
            #r_petrosian = f(eta)

            idx_list = np.where(petrosian_list <= eta)[0]

            #print(total_flux)
            epsilon = r_99 / r_petrosian
            p_corr = None#Petrosian(r_list, area_arr, flux_arr, epsilon=epsilon)

            p_corr_eta = eta
            p_corr_epsilon = epsilon
            p_corr_r_petrosian = r_petrosian
            p_corr_r_total_flux = r_petrosian * p_corr_epsilon
            f = interp1d(r_list, flux_arr, kind='linear')
            p_corr_total_flux = f(p_corr_r_total_flux)
            p_corr_r_20 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.2)
            p_corr_r_80 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.8)
            p_corr_r_50 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.5)
            p_corr_r_90 = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.9)
            p_corr_r_half_light = fraction_flux_to_r(r_list, flux_arr, p_corr_r_total_flux, fraction=0.5)


            p_uncorr_eta = eta
            p_uncorr_epsilon = 2
            p_uncorr_r_petrosian = r_petrosian
            p_uncorr_r_total_flux = r_petrosian * p_uncorr_epsilon
            f = interp1d(r_list, flux_arr, kind='linear')
            p_uncorr_total_flux = f(p_uncorr_r_total_flux)
            p_uncorr_r_20 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.2)
            p_uncorr_r_80 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.8)
            p_uncorr_r_50 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.5)
            p_uncorr_r_90 = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.9)
            p_uncorr_r_half_light = fraction_flux_to_r(r_list, flux_arr, p_uncorr_r_total_flux, fraction=0.5)

            row = [
                n, r_eff, amplitude, total_flux, r_99, r_20, r_80, r_50, r_90, 5 *np.log10(r_80 / r_20), 5 *np.log10(r_90 / r_50), c_petro_0502,
                p_corr_eta, p_corr_epsilon, p_corr_r_petrosian, p_corr_r_half_light, p_corr_r_total_flux, p_corr_total_flux, p_corr_r_20, p_corr_r_80, p_corr_r_50, p_corr_r_90, 5 *np.log10(p_corr_r_80 / p_corr_r_20), 5 *np.log10(p_corr_r_90 / p_corr_r_50), 
                p_uncorr_eta, p_uncorr_epsilon, p_uncorr_r_petrosian, p_uncorr_r_half_light, p_uncorr_r_total_flux, p_uncorr_total_flux, p_uncorr_r_20, p_uncorr_r_80, p_uncorr_r_50, p_uncorr_r_90, 5 *np.log10(p_uncorr_r_80 / p_uncorr_r_20), 5 *np.log10(p_uncorr_r_90 / p_uncorr_r_50),
            ]
            assert not np.any(np.isnan(np.array(row))), "{}".format(n)
            #plt.plot(r_list, petrosian_list)
            #plt.show()
            del petrosian_list
            del flux_arr
            del r_list
            
            rows.append(row)
            #bar.update()
            
    #print("done")
    print(pid,':' , n)
    return Table(rows=rows, names=names)
    
plt.show()


In [ ]:
run = False
processes = 6 #mp.cpu_count
file_name = 'pure_grid_fine_petro_const_3.csv' 
file_name = 'fine_petro_table_no_psf.csv'

if run:
    #Start multiprocessing photometry (a core per filter)
    if __name__ == '__main__':
        with Pool(processes=processes) as pool: 
            print("Starting MP")
            try:
                grid_tables = pool.map(process_image, list(n_list))
            except Exception as e:
                pool.close()
                raise e 
            pool.close()
            
        t = vstack(grid_tables)
        print("Done")
        
        if file_name:
            t.write(file_name, overwrite=True)
        
else:
    t = Table.read(file_name)

In [ ]:
# file_name = 'fine_petro_table_no_psf.csv'
# t.write(file_name, overwrite=True)

In [ ]:
# t.write('pure_grid_fine.csv')#, overwrite=True)
#t = Table.read('pure_grid.csv')

In [ ]:
# t = Table.read('pure_grid_fine_petro_const_3.csv')

# Fit Models

## Corrected vs Uncorrected Concentration Index

In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['uncorr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.SmoothlyBrokenPowerLaw1D()

line_init =  models.Polynomial1D(6)

fitted_line = fit(line_init, x, y)

c_relation = fitted_line

print(fitted_line.param_names)
print(fitted_line.parameters)

arr = np.arange(min(x), max(x)+0.1, 0.1)

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.title("Corrected vs Uncorrected Concentration Index")
plt.xlabel("Uncorrected C2080")
plt.ylabel("Corrected C2080")
plt.legend()

savefig("corr_vs_uncorr_c2080.png")

In [ ]:
plt.scatter(x, (y-fitted_line(x))/y, c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.ylim(-.5, .5)
plt.axhline(-.05)
plt.axhline(.05)
plt.show()

In [ ]:
print(repr(c_relation))

## Concentration Index vs Sérsic Index

In [ ]:
max_n = 15
x = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['sersic_n'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.PowerLaw1D()

line_init =  models.Polynomial1D(5)

fitted_line = fit(line_init, x, y)#, maxiter=100000, acc=0)

print(fitted_line.param_names)
print(fitted_line.parameters)

arr = np.arange(min(x), max(x), 0.1)

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.title("Corrected Concentration Index vs Sérsic Index")
plt.xlabel("Corrected C2080")
plt.ylabel("Sérsic Index n")
plt.legend()

In [ ]:

plt.scatter(y, (y-fitted_line(x))/y, c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.ylim(-.5, .5)
plt.axhline(-.05)
plt.axhline(.05)
plt.show()

In [ ]:
print(repr(fitted_line))
c2080_to_n = fitted_line

## Epsilon vs Sérsic Index

In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['sersic_n']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
#fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.BrokenPowerLaw1D()

line_init = models.Polynomial1D(5) + exp1d 
fitted_line = fit(line_init, x, y, maxiter=10000, acc=0)

# x = list(t[np.where(t['sersic_n'] > 15)]['sersic_n']) 
# y = list(t[np.where(t['sersic_n'] > 15)]['corr_epsilon'])

# x = np.array(x)
# y = np.array(y)


print(repr(fitted_line))
print(fitted_line.parameters)

arr = np.array(list(np.arange(min(x), max(x), 0.1))+[max(x)])

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

# plt.scatter(x, y, marker='o', c='b', label='data')

# plt.plot(x, models.Polynomial1D(4, c0=-5.09265992, c1=-1.5511942, c2=-0.28384006, c3=-0.00909155, c4=-0.00285897,)(x))
# plt.plot(x, models.Exponential1D(6.00314167, tau=2.62515105)(x))

plt.title("Epsilon vs Sérsic Index")
plt.xlabel("Sérsic Index n")
plt.ylabel("Epsilon")
plt.legend()


In [ ]:

plt.scatter(x, (y-fitted_line(x))/y, c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.ylim(-.5, .5)
plt.axhline(-.05)
plt.axhline(.05)
plt.show()

In [ ]:
print(repr(fitted_line))
n_to_epsilon = fitted_line

In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

#x = np.log10((np.exp(x/5))**-1)*5

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
#fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D(tau=1)
power = powerlaws.BrokenPowerLaw1D()

line_init =  exp1d + models.Polynomial1D(6)
fitted_line = fit(line_init, x, y, maxiter=1000000, acc=0)

# x = list(t[np.where(t['sersic_n'] > 15)]['sersic_n']) 
# y = list(t[np.where(t['sersic_n'] > 15)]['corr_epsilon'])

# x = np.array(x)
# y = np.array(y)


print(repr(fitted_line))
print(fitted_line.parameters)

arr = np.array(list(np.arange(min(x), max(x), 0.1))+[max(x)])

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

# plt.scatter(x, y, marker='o', c='b', label='data')

# plt.plot(x, models.Polynomial1D(4, c0=-5.09265992, c1=-1.5511942, c2=-0.28384006, c3=-0.00909155, c4=-0.00285897,)(x))
# plt.plot(x, models.Exponential1D(6.00314167, tau=2.62515105)(x))

plt.title("Sérsic Index vs Epsilon")
plt.xlabel("Sérsic Index n")
plt.ylabel("Epsilon")
plt.legend()


In [ ]:

plt.scatter(x, (y-fitted_line(x))/y, c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.ylim(-.5, .5)
plt.axhline(-.05)
plt.axhline(.05)
plt.show()

## corr_epsilon vs  sersic_petro_0502

In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['sersic_petro_0502']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

#x = np.log10((np.exp(x/5))**-1)*5

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
#fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.BrokenPowerLaw1D()

line_init =    models.Polynomial1D(6) + exp1d
fitted_line = fit(line_init, x, y, maxiter=1000000, acc=0)

# x = list(t[np.where(t['sersic_n'] > 15)]['sersic_n']) 
# y = list(t[np.where(t['sersic_n'] > 15)]['corr_epsilon'])

# x = np.array(x)
# y = np.array(y)


print(repr(fitted_line))
print(fitted_line.parameters)

arr = np.array(list(np.arange(min(x), max(x), 0.1))+[max(x)])

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

# plt.scatter(x, y, marker='o', c='b', label='data')

# plt.plot(x, models.Polynomial1D(4, c0=-5.09265992, c1=-1.5511942, c2=-0.28384006, c3=-0.00909155, c4=-0.00285897,)(x))
# plt.plot(x, models.Exponential1D(6.00314167, tau=2.62515105)(x))

plt.title("Epsilon vs $P_{0502}$")
plt.xlabel("$P_{0502}$")
plt.ylabel("Epsilon")
plt.legend()


In [ ]:

plt.scatter(x, (y-fitted_line(x))/y, c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.ylim(-.5, .5)
plt.axhline(-.05)
plt.axhline(.05)
plt.show()

In [ ]:
plt.plot(t['sersic_n'], np.exp(t['sersic_petro_0502']/5)*t['corr_r_petrosian'])
plt.axvline(15)

# Missing Mag

In [ ]:
@custom_model
def missing_mag(x, p1=5.1e-4, p2=1.451, c=0):
    return p1*np.exp(x**p2) + c

In [ ]:
max_n = 10
idxs = np.where(t['sersic_n'] <=max_n)

x = t['uncorr_r_90'][idxs] / t['uncorr_r_50'][idxs]
y = (-2.5*np.log10(t['uncorr_total_flux'][idxs])) - (-2.5*np.log10(t['corr_total_flux'][idxs]))
z = missing_mag()(x)
n = t['sersic_n'][idxs]

plt.scatter(n, y, c='black')
plt.plot(n, z, c='r')
plt.show()

plt.scatter(n, (z-y), c='r')

plt.axhline(0.01)
plt.axhline(-0.01)


plt.show()

###################################################



In [ ]:
x = t['uncorr_r_90'][idxs] / t['uncorr_r_50'][idxs]

line_init = missing_mag()
line_init = models.Exponential1D(amplitude=3) + models.Polynomial1D(3)

fit = fitting.LevMarLSQFitter()
fitted_line = fit(line_init, x, y, maxiter=100000, acc=1e-07,)
print(fitted_line)

plt.scatter(n, y, c='black')
plt.plot(n, fitted_line(x), c='r')

print(" ")
print(repr(fitted_line))

plt.show()

plt.scatter(n, (fitted_line(x)-y), c='r')

plt.axhline(0.01)
plt.axhline(-0.01)


plt.show()

In [ ]:
x = t['uncorr_r_80'][idxs] / t['uncorr_r_20'][idxs]

line_init = missing_mag()
line_init = models.Exponential1D() + models.Polynomial1D(3)

fit = fitting.LevMarLSQFitter()
fitted_line = fit(line_init, x, y, maxiter=10000, acc=1e-07,)
print(fitted_line)

plt.scatter(n, y, c='black')
plt.plot(n, fitted_line(x), c='r')

print(" ")
print(repr(fitted_line))
plt.show()

plt.scatter(n, (fitted_line(x)-y), c='r')

plt.axhline(0.01)
plt.axhline(-0.01)


plt.show()

In [ ]:
x = t['uncorr_c2080'][idxs]

line_init = missing_mag()
line_init = models.Exponential1D(1) + models.Polynomial1D(3)

fit = fitting.LevMarLSQFitter()
fitted_line = fit(line_init, x, y, maxiter=10000, acc=1e-07,)
print(fitted_line)

plt.scatter(n, y, c='black')
plt.plot(n, fitted_line(x), c='r')

print(" ")
print(repr(fitted_line))
plt.show()

plt.scatter(n, (fitted_line(x)-y), c='r')

plt.axhline(0.01)
plt.axhline(-0.01)


plt.show()

# Plots

In [ ]:
x = list(t[np.where(t['sersic_n'] < 15)]['uncorr_c2080']) 
y = list(t[np.where(t['sersic_n'] < 15)]['corr_c2080'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = c_relation 

print(fitted_line.param_names)
print(fitted_line.parameters)

arr = np.arange(min(x), max(x)+0.1, 0.1)

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model'.format(*fitted_line.parameters))

plt.title("Corrected vs. Uncorrected Concentration Index")
plt.xlabel("Uncorrected C2080")
plt.ylabel("Corrected C2080")
plt.legend()

savefig("corr_vs_uncorr_c2080.png")

In [ ]:
figsize = [10*2., 10*1]
fig, ax = plt.subplots(1, 2, figsize=figsize)

plt.sca(ax[0])

max_n = 15
x = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['sersic_n'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = c2080_to_n


plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model')

plt.title("Sérsic Index vs. Corrected Concentration Index")
plt.xlabel("Corrected C2080")
plt.ylabel("Sérsic Index n")
plt.legend()

###########################################################

plt.sca(ax[1])

x = list(t[np.where(t['sersic_n'] < 15)]['uncorr_c2080']) 
y = list(t[np.where(t['sersic_n'] < 15)]['sersic_n'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = lambda x: c2080_to_n(c_relation(x))


plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model')

plt.title("Sérsic Index vs. Uncorrected Concentration Index")
plt.xlabel("Uncorrected C2080")
plt.ylabel("Sérsic Index n")
plt.legend()


savefig("c2080_vs_n.png")

In [ ]:
max_n = 15

figsize = [10*3., 10*1]
fig, ax = plt.subplots(1, 3, figsize=figsize)

plt.sca(ax[0])


print('corr_c2080')
x = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = lambda x: n_to_epsilon(c2080_to_n(x))

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model')

plt.title("Epsilon vs. Corrected Concentration Index")
plt.xlabel("Corrected C2080")
plt.ylabel("Epsilon")
plt.legend()

###########################################################

plt.sca(ax[1])
print('uncorr_c2080')
x = list(t[np.where(t['sersic_n'] < max_n)]['uncorr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = lambda x: n_to_epsilon(c2080_to_n(c_relation(x)))

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model')

plt.title("Epsilon vs. Unorrected Concentration Index")
plt.xlabel("Unorrected C2080")
plt.ylabel("Epsilon")
plt.legend()


###########################################################

plt.sca(ax[2])
print('sersic_n')
x = list(t[np.where(t['sersic_n'] < max_n)]['sersic_n']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

fitted_line = n_to_epsilon

print(fitted_line.param_names)
print(fitted_line.parameters)

arr = np.array(list(np.arange(min(x), max(x), 0.1))+[max(x)])

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Model'.format(*fitted_line.parameters))

plt.title("Epsilon vs. Sérsic Index")
plt.xlabel("Sérsic Index n")
plt.ylabel("Epsilon")
plt.legend()

savefig("epsilon_vs_n_and_c2080.png")
plt.show()


In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['sersic_petro_0502']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['corr_epsilon'])

x = np.array(x)
y = np.array(y)

#x = np.log10((np.exp(x/5))**-1)*5

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
#fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.BrokenPowerLaw1D()

line_init =    models.Polynomial1D(6) + exp1d
fitted_line = fit(line_init, x, y, maxiter=1000000, acc=0)

# x = list(t[np.where(t['sersic_n'] > 15)]['sersic_n']) 
# y = list(t[np.where(t['sersic_n'] > 15)]['corr_epsilon'])

# x = np.array(x)
# y = np.array(y)


print(repr(fitted_line))
print(fitted_line.parameters)

arr = np.array(list(np.arange(min(x), max(x), 0.1))+[max(x)])

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

# plt.scatter(x, y, marker='o', c='b', label='data')

# plt.plot(x, models.Polynomial1D(4, c0=-5.09265992, c1=-1.5511942, c2=-0.28384006, c3=-0.00909155, c4=-0.00285897,)(x))
# plt.plot(x, models.Exponential1D(6.00314167, tau=2.62515105)(x))

plt.title("Epsilon vs $P_{0502}$")
plt.xlabel("$P_{0502}$")
plt.ylabel("Epsilon")
plt.legend()


savefig("epsilon_vs_P0502.png")
plt.show()

In [ ]:
figsize = [10*2., 10*1]
fig, ax = plt.subplots(1, 2, figsize=figsize)

plt.sca(ax[0])

max_n = 15
idxs = np.where(t['sersic_n'] <=15)

plt.scatter(t['sersic_n'][idxs], t['uncorr_total_flux'][idxs]/t['corr_total_flux'][idxs], c='black')

plt.axvline(4, linestyle='--', label='(n=4, fraction=0.838)', c='b')
plt.axhline(0.838, linestyle='--', c='b')

plt.xlabel('Sérsic Index n')
plt.ylabel('Fraction of Total Flux Recoved')
plt.title('Fraction of total flux recoved by $\epsilon = 2$')
plt.legend()



plt.sca(ax[1])

max_n = 15
idxs = np.where(t['sersic_n'] <=15)

x = t['uncorr_c2080'][idxs]
y = (-2.5*np.log10(t['uncorr_total_flux'][idxs])) - (-2.5*np.log10(t['corr_total_flux'][idxs]))
z = missing_mag()(x)
n = t['sersic_n'][idxs]

line_init = missing_mag()
line_init = models.Exponential1D() + models.Polynomial1D(3)

fit = fitting.LevMarLSQFitter()
fitted_line = fit(line_init, x, y, maxiter=10000, acc=1e-07,)

plt.scatter(n, y, c='black', label='data')
plt.plot(n, fitted_line(x), c='r', linewidth=2, label='Fitted Model'.format(*fitted_line.parameters))


plt.title("$\Delta$m vs Uncorrected $C_{2080}$")
plt.xlabel("Uncorrected $C_{2080}$")
plt.ylabel("$\Delta$m")
plt.legend()


savefig("fraction_of_total_flux_recoved.png")

In [ ]:
fitted_line

In [ ]:
r_eff=25
figsize = [10*3., 10*3]
fig, ax = plt.subplots(3, 3, figsize=figsize)

r_list_half = np.arange(0, 51, 1)

r_list = list(-r_list_half) + list(r_list_half)
r_list = np.array(list(sorted(np.unique(np.array(r_list)))))


for i, n in enumerate([0.5, 1, 4]):
    plt.sca(ax[0, i])
    b_n = gammaincinv(2. * n, 0.5)
    amplitude=1 /np.exp(b_n)

    model = models.Sersic1D(amplitude, r_eff=r_eff, n=n)


    i_list =  model(abs(r_list))
    plt.plot(r_list, i_list, c='black', linewidth=4, label="Sersic Profile".format(n))
    #plt.fill_between(r_list, i_list, color='b', alpha=0.1)
    #plt.bar(r_list, i_list, alpha=0.3)
    
    psf = models.Moffat1D(amplitude=1, gamma=1, alpha=0.5)(r_list)
    psf = psf/psf.sum()
    con_list = convolve(i_list, psf)

    plt.plot(r_list, con_list, c='r', linewidth=6, label="Moffat Convolved".format(n))
    #plt.fill_between(r_list, con_list, color='r', alpha=0.3)
    #plt.bar(r_list, con_list, width=0.6, color='r', alpha=0.3)
    
    plt.xlim(-50,50)
    plt.title("Sérsic Profile n = {}".format(n))
    plt.xlabel("Radius r")
    plt.ylabel("I(r)")
    plt.legend(loc='upper left')
    
    plt.sca(ax[1, i])
    half_idx = r_list_half.size-1
    
    f_list = [sum(i_list[half_idx: half_idx+i]) for i in range(half_idx+1)]
    f_con_list = [sum(con_list[half_idx: half_idx+i]) for i in range(half_idx+1)]
    
    plt.plot(r_list_half, f_list, c='black', linewidth=4, label="Sersic Profile".format(n))
    plt.plot(r_list_half, f_con_list, c='r', linewidth=6, label="Moffat Convolved".format(n))
    #plt.fill_between(r_list_half, f_con_list, color='r', alpha=0.3)
    
    plt.title("Curve of Growth n = {}".format(n))
    plt.xlabel("Radius r")
    plt.ylabel("L(<r)")
    plt.legend(loc='upper left')
    
    plt.sca(ax[2, i])
    
    area = [np.pi*i**2 for i in range(half_idx+1)]
    
    p = calculate_petrosian(area, f_list)
    p_con = calculate_petrosian(area, f_con_list)
    
    plt.plot(r_list_half, p, c='black', linewidth=4, label="Sersic Profile".format(n))
    plt.plot(r_list_half, p_con, c='r', linewidth=6, label="Moffat Convolved".format(n))
    #plt.fill_between(r_list_half, p_con, color='r', alpha=0.3)
    
    plt.title("Petrosian n = {}".format(n))
    plt.xlabel("Radius r")
    plt.ylabel("$\eta$(r)")
    plt.legend(loc='upper right')
    
    
    
    
savefig("psf_effect_sersic.png")

# Other

In [ ]:
@custom_model
def f(c, k=2, a=1):
    return (c/k)**-a
    return amp*(lambertw(c*np.log10(k)) / np.log10(k)).astype(float)

In [ ]:
max_n = 15

x = list(t[np.where(t['sersic_n'] < max_n)]['corr_c2080']) 
y = list(t[np.where(t['sersic_n'] < max_n)]['sersic_n'])

x = np.array(x)
y = np.array(y)

plt.scatter(x, y, marker='o', c='black', label='data')

# initialize a linear fitter
fit = fitting.LevMarLSQFitter()
#fit = fitting.LinearLSQFitter()

# initialize a linear model
exp1d = models.Exponential1D()
power = powerlaws.BrokenPowerLaw1D() 

#line_init = power + models.Const1D()
#fitted_line = fit(line_init, x, y, maxiter=100000, acc=0)

print(fitted_line.param_names)
print(fitted_line.parameters)

arr = np.arange(min(x), max(x), 0.1)

plt.plot(x, fitted_line(x), c='red', linewidth=2,
         label='Fitted Model'.format(*fitted_line.parameters))

plt.title("Corrected Concentration Index vs Sérsic Index")
plt.xlabel("Corrected C2080")
plt.ylabel("Sérsic Index n")
plt.legend()